# 準備
安裝必要工具、載入所需資料

## 版本確認

In [ ]:
!python --version

Python 3.10.12


## 安裝工具

In [1]:
# !pip install -U sentence-transformers
# !pip install python-dotenv

!pip install openai
!pip install langchain

# document load
# Word
# !pip install --upgrade --quiet  docx2txt # 選擇一：load 為純文字
# !pip install unstructured # 選擇二：處理非結構化文件，使用 UnstructuredWordDocumentLoader 會用到
# !pip install python-docx # 用於讀取和寫入 Microsoft Word DOCX 文件的 lib (UnstructuredWordDocumentLoader 會依賴它，需要安裝)

# embedding
# 專為中文設計的斷詞套件
# !pip install jieba # 號稱最強中文斷詞處理，中國開發。先測試，之後再考慮是否要用中研院的 (要申請才能使用)
# 通用型套件
# !pip install -U langchain-openai # OpenAI
!pip install -U langchain-nomic # LangChain Nomic embedding

# vector store
!pip install langchain-chroma # 輕量，且在記憶體儲存的向量儲存庫

# retriever
!pip install lark # self-query retriever 所需套件

# llm
!pip install langchain-groq # groq 服務


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packagi

In [ ]:
# 下載 jieba 繁中詞庫
# !wget -nc https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big

--2024-05-03 03:25:47--  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8583143 (8.2M) [text/plain]
Saving to: ‘dict.txt.big’

dict.txt.big        100%[===================>]   8.18M  --.-KB/s    in 0.05s   

2024-05-03 03:25:48 (153 MB/s) - ‘dict.txt.big’ saved [8583143/8583143]



## 設定 API KEY

In [ ]:
# 設定 API KEY
import os

### Nomic Embedding

In [56]:
os.environ['NOMIC_API_KEY'] = 'nk-c6FfUDi2q62RevmsMMHgjbhShKWEaeBKf1Ad6upZxRI'

### LangChang
使用 LangSmith 做 evaluation 時，需要 LangChang 的 API KEY

In [5]:
os.environ['LANGCHAIN_API_KEY'] = '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'

### GROQ
能使用 groq 服務 (有使用限制) 中的 llama3

In [6]:
os.environ['GROQ_API_KEY'] = "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"
groq_api_key = os.environ['GROQ_API_KEY']

# VectorDB

## 連線設定

In [7]:
'''
炳勛創建並使用遠端的 chromadb 連線。
> 創建的方法記在 Notion (https://www.notion.so/Chroma-bb88caeac5154426beeff4726be4a208?pvs=4 粗略記一下，有空再整理）
> 可以利用以下代碼進行連線
'''
# 可以更改 collection name 創建屬於自己的資料表

from langchain_community.vectorstores import Chroma # 輕量，且在記憶體儲存的向量儲存庫
import chromadb
from chromadb.config import Settings

#連線設定
httpClient = chromadb.HttpClient(
  host='64.176.47.89', port=8000,
  settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="admin:admin")
  )

## DB 操作

In [ ]:
collections = httpClient.list_collections() # 資料庫列表

In [ ]:
# collection = httpClient.create_collection("X131010") # 創建資料庫 (用 OpenAIEmbedding 方法)

In [ ]:
# httpClient.delete_collection(name="X131010") # 刪除資料庫

In [8]:
collections = httpClient.list_collections() # 資料庫列表
collections

[Collection(name=xt131028),
 Collection(name=X131010),
 Collection(name=my_collection)]

# Llama3

## method1: LLM Client Setting
使用自行架設 LLM 的 API 服務，透過 LangChain 的 OpenAI  連接。

In [33]:
from openai import OpenAI
llm_client = OpenAI(
    base_url="https://34d2-140-109-17-42.ngrok-free.app/v1/", # "http://<Your api-server IP>:port" # AUA 學校 primehub 上的 llama3 (加掛 gpu)
    api_key = "sk-no-key-required" # 並沒有使用真正的 OpenAI API，可以隨意填寫
    # max_tokens = # 規範模型回答的長度
)

### 連線測試

In [6]:
# 用如下 stream 方式打 api 連自建的 llama3，進行 QA 測試
completion = llm_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "你是一個旅遊達人, 會很多旅遊相關的知識, 而且用 zh_TW 回答"},
        {"role": "user", "content": "我想去阿里山遊玩, 可以介紹我景點嗎?"}
    ],
    stream=True
)
#print(completion.choices[0].message)
for chunk in completion:
    #print(chunk)
    print(chunk.choices[0].delta.content, end ="")
    #print("**")

阿里山是一個非常美麗的旅遊景點！阿里山是位於台灣南投縣的山區，擁有豐富的自然資源和文化資產。以下是一些阿里山的主要景點：

1. 阿里山森林遊樂區：這是阿里山的主要景點之一，提供了多種遊戲和活動，包括森林步道、山林步道、溪流步道等。
2. 阿里山森林鐵路：這是一條穿過阿里山的鐵路，提供了美麗的景色和遊戲活動。
3. 阿里山森林遊樂區的遊戲區：這裡提供了多種遊戲和活動，包括滑草、滑雪、騎馬等。
4. 阿里山森林遊樂區的餐廳：這裡提供了多種美食和飲料，包括台灣的傳統美食和西式美食。
5. 阿里山森林遊樂區的住宿：這裡提供了多種住宿選擇，包括旅館、民宿和露營等。

這些景點都是阿里山的主要景點之一，提供了多種遊戲和活動，讓遊客可以在阿里山度過一個美好的旅遊體驗。None

## method2: 使用 groq 服務
primehub 上的 llama3 會失效
暫時先改用 groq 的服務 (有使用限制)

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [11]:
chat_groq = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-70b-8192")
# MODEL_NAME = "llama3-70b-8192" or "llama3-8b-8192"
# 70b 才支援中文

### 使用測試

In [16]:
system = "You are a helpful assistant response with Chinese-Traditional (Taiwan)."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat_groq
chain.invoke({"text": "什麼是大型語言模型？"})

AIMessage(content='\n\n大型語言模型（Large Language Model，LLM）是一種人工智慧（AI）模型，能夠處理和理解大量的語言資料。這些模型通常是基於深度學習（Deep Learning）技術，使用大量的文本資料進行訓練，以學習語言的模式和規律。\n\n大型語言模型的特點包括：\n\n1. 大量參數：LLM 通常具有數億個參數，能夠捕捉到語言的複雜規律。\n2. 深度神經網絡：LLM 使用深度神經網絡（Deep Neural Network）來處理語言資料，能夠學習到語言的抽象表示。\n3. 大量訓練資料：LLM 需要大量的文本資料進行訓練，以學習語言的規律和模式。\n4. 多任務能力：LLM 能夠處理多種語言任務，例如文本生成、語言翻譯、文本分類等。\n\n大型語言模型的應用包括：\n\n1. 文本生成：LLM 能夠生成高質量的文本，例如文章、新聞、書籍等。\n2. 語言翻譯：LLM 能夠進行高質量的語言翻譯，例如翻譯文章、網頁等。\n3. 文本分類：LLM 能夠對文本進行分類，例如垃圾郵件過濾、情感分析等。\n4. 對話系統：LLM 能夠用於對話系統，例如客服聊天機器人、語音助手等。\n\n總之，大型語言模型是人工智慧領域的一個重要進展，能夠處理和理解大量的語言資料，並應用於多種領域。', response_metadata={'token_usage': {'completion_time': 1.371, 'completion_tokens': 417, 'prompt_time': 0.017, 'prompt_tokens': 38, 'queue_time': None, 'total_time': 1.388, 'total_tokens': 455}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_87cbfbbc4d', 'finish_reason': 'stop', 'logprobs': None}, id='run-07b88320-2425-412c-937e-2d4e4f065b5b-0')

# Chat

In [18]:
query = "全域技術班課程開始日期？"

## 指定使用向量資料庫

### nomic embedding db

In [ ]:
# 改用 nomic embdding 服務 (需要先申請 api key: https://atlas.nomic.ai)
from langchain_nomic.embeddings import NomicEmbeddings
nomic_embedding = NomicEmbeddings(model="nomic-embed-text-v1.5")

In [73]:
# tell LangChain to use our client and collection name
db4 = Chroma(
    client = httpClient,
    collection_name = "xt131028",
    embedding_function = nomic_embedding
)
print(db4.get()) # 查看所有資料
print(db4._collection.count())

{'ids': ['00cdee34-7e24-4eb3-9942-d629b5fce191', '028fd2f4-16d3-424d-b7fe-7550f7054f45', '04224a16-e383-492f-a8a8-7c2f4ae41456', '04f828b4-daca-450c-981a-3e78bec960ed', '06218b86-7dfb-42b8-bbbc-03f9c19fa137', '0675e3bd-efee-4e1e-b019-620f67bd1149', '06b7c77a-026f-46ea-a710-cbb6b264cd32', '07a54ccf-d3d6-4852-8c9c-94682087e8be', '07f1a520-f1aa-4bc5-a641-9b4a3e4a36fe', '0816627b-5f39-48fc-af5b-fef7483d8465', '08986553-8aa0-454f-90f0-74e4b11272b5', '08f8fc22-e06e-4a94-a335-a5997f338d3c', '095ab84e-24a9-4e21-81ed-127c284e352a', '0a998311-9ce4-435a-843e-400b4ba1c5e8', '0bd59544-db0e-4d90-a9f9-714cb3344120', '0c0c5a08-625a-4ba1-8d45-de0083d3573e', '0c6a909b-0221-402f-b0e5-591c8d5ac826', '0d9bc5d1-9728-4d32-868c-a2e82a9e8577', '0e6ba2ea-d67d-43ad-b808-8e73cd2aed6d', '0eef8e9c-6a1b-4f33-b108-1e3fa9fc130a', '128aaf04-a7a1-42f1-a546-8949d36cf8b6', '1714f4e9-054a-4fc2-8efa-31451f5d1575', '17998496-b59d-4144-9152-3da807dbe803', '18dee838-50b9-4868-83c8-0a4430a00c9c', '19349bf9-cd5f-42d9-b13e-224ac8

### similar search 測試

In [ ]:
query = "全域技術班課程開始日期？"
documents = db4.similarity_search(query)
print(documents[0].page_content)

## Evaluation

In [62]:
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA

In [63]:
# llm = OpenAI(temperature = 0.0) # 自建 llm 不會串，暫用 OpenAI 的
llm = chat_groq # 使用 groq 上的 llama3-70b

qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever = db4.as_retriever(search_type = "mmr"),
    chain_type = "map_reduce",
    verbose=True,
    # chain_type_kwargs = {
    #     "document_separator": "<<<<>>>>>"
    # }
)

### LLM-Generated examples
利用 LLM 自動化生成 (自動從每個文檔建立一個 Q, A pair

In [64]:
from langchain.evaluation.qa import QAGenerateChain

In [65]:
example_gen_chain = QAGenerateChain.from_llm(llm)

In [66]:
data = db4.get()['documents'] # 取原始文本 chunks
print(type(data))
print(data)

<class 'list'>
['常見問題: 請問你們有學費補助嗎？因為我看我們醫院有補助計畫，那如果你們這一期沒有，我是否可以轉下一期？\n回答: 我們學校沒有提供學費補助，應該是貴醫院單位的補助計畫，可能需要您與貴單位做內部確認，是否能申請院內補助，我本校無權涉入。', '台北總校第二十期經理人週末研修班招生簡章: 8 日起，且未達全期三分之一期間內提出退費申請者( 2024/07/14 ~ 2024/08/08 )應退還當期開班約定繳納費用總額百分之五十。於當期課程開課期間已達全期三分之一以上提出退費申請者( 2024/08/09～ )所收取之當期開班約定繳納費用得全數不予退還。備註:本表所稱「開課日」、「全期三分之一」等日期，皆以本校當期課程行事曆規定之日期為判斷依據。課程大綱立即報名', 'AIGC 實戰冬令營：高中生的第一個生成式 AI 營隊: 天內 完成註冊繳費。請於規定時間內辦理註冊及繳費，繳費方式可選擇線上金流 (刷卡) 或非線上金流 (轉帳)。若選擇非線上金流，系統會產生一組虛擬帳號，請務必在繳費期限內完成匯款繳費。繳費後才算完成報名程序。未依規定辦理或逾期未註冊者，取消資格，事後不得以任何理由要求補註冊。講師陣容手機螢幕建議使用水平橫式瀏覽課程名稱姓名職稱講師簡介線上預習從0到1的Prompt之旅：開啟AI的無限可能簡光正鉅鋼機械股份有限公司 數位創新中心經理現任職於台中市鉅鋼機械-數位創新中心經理。主要負責企業數位轉型、敏捷文化、AI與智慧製造相關專案，並於2022年榮獲哈佛商業評論舉辦之數位轉型鼎革獎-楷模獎之殊榮。熱愛研究與分享IT新領域之技術，在AI、雲端架構、區塊鏈、工業物聯網等領域均有研究。大型語言模型 (LLM) 概論蔡政霖台灣人工智慧學校 AI 工程師台灣人工智慧學校台北總校 AI工程師陽明交通大學生醫資訊所博士候選人研究領域為醫學影像分割最近新增 Prompt Engineering 技能AI', '大型語言模型實作初階班 (LLM-A) 招生簡章: 班報名皆為候補名額)辦公室流程自動化結合大語言模型，讓您的工作更輕鬆、更高效您是否曾經為了重複性的辦公室工作而感到煩悶？您是否想要利用最新的人工智慧技術來提升您的工作品質和效率？如果您的答案是肯定的，請一定要了解「辦公室流程自動化」和「大型語言模型」協作的優勢。「辦公室流程自動化

In [67]:
# the warning below can be safely ignored

In [68]:
# 自動從每個文檔的每個 chunks 建立一個 QA pair
examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:]]
)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [70]:
print(type(examples[0]))
examples[0]['qa_pairs']
# qa_examples = examples['qa_pairs']
# print(len(examples))

<class 'dict'>


{'query': 'What is the response to the question of whether the school provides tuition fee assistance?',
 'answer': 'The school does not provide tuition fee assistance, and any assistance is likely to be a internal plan of the hospital unit, which requires internal confirmation.'}

In [71]:
# 因為建立的 examples 會是兩層的 dictionary list, 只組合第二層的 dictionary 作為最後的 qa_examples list
qa_examples = []
for example in examples:
  qa_examples.append(example['qa_pairs'])
print(qa_examples)
print(len(qa_examples))

[{'query': 'What is the response to the question of whether the school provides tuition fee assistance?', 'answer': 'The school does not provide tuition fee assistance, and any assistance is likely to be a internal plan of the hospital unit, which requires internal confirmation.'}, {'query': 'If a student applies for a refund within the first one-third of the course period (2024/07/14 ~ 2024/08/08), what percentage of the tuition fee will be refunded?', 'answer': '50%'}, {'query': 'What is the title of the course that the lecturer, 蔡政霖, will be teaching, and what is his research focus?', 'answer': 'The title of the course is "從0到1的Prompt之旅：開啟AI的無限可能" and his research focus is on medical image segmentation and Prompt Engineering.'}, {'query': 'What are some examples of tasks that can be automated through "辦公室流程自動化" (office process automation)?', 'answer': '填寫表單、發送電子郵件、處理文件、產生報告等 (Filling out forms, sending emails, processing files, generating reports, etc.)'}, {'query': 'What is the tit

In [72]:
len(data)

227

### LLM assisted evaluation
利用 LLM 完成自動化評估

In [ ]:
predictions = qa_chain_mr.apply(qa_examples) # 對所有 QA 範例進行預測，以供後續評估



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA ch

In [ ]:
print(len(predictions)) # 確認數量

#### QAEvalChain
評估結果只會是 CORRECT 或 INCORRECT

In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
# 對所有 QA 範例和預測結果進行比對評估，返回一堆評估分級
graded_outputs = eval_chain.evaluate(qa_examples, predictions)
len(graded_outputs)

In [ ]:
graded_outputs[0]

In [ ]:
for i, eg in enumerate(qa_examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted " + graded_outputs[i]['results'])
    print()

In [ ]:
graded_outputs[0]

#### Scoring Evaluator

請注意，對於 LLM 中的任何特定等級都應該持保留態度。得分為「8」的預測可能不會比得分為「7」的預測好多少。

In [ ]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("labeled_score_string", llm=llm)

In [ ]:
# Sample test
eval_result = evaluator.evaluate_strings(
    prediction="You can find them in the dresser's third drawer.",
    reference="The socks are in the third drawer in the dresser",
    input="Where are my socks?",
)
print(eval_result['score'])

In [ ]:
# 對所有 QA 範例和預測結果進行比對評估，返回一堆評估分級
# graded_outputs = eval_chain.evaluate(qa_examples, predictions)
# len(graded_outputs)

for i, eg in enumerate(qa_examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Score: " +
          str(
              evaluator.evaluate_strings(
                  prediction=predictions[i]['result'],
                  reference=predictions[i]['answer'],
                  input=predictions[i]['query']
                  )['score']
              )
          )
    print()

# eval_result =
# print(eval_result)

### LangSmith
https://docs.smith.langchain.com/evaluation/quickstart

(研究中)

#### intall LangSmith

In [ ]:
!pip install -U langsmith # evaluation platform

#### Evalutaion

In [ ]:
from langsmith import Client
from langsmith.schemas import Run, Example
from langsmith.evaluation import evaluate
import openai
from langsmith.wrappers import wrap_openai

client = Client()

# Define dataset: these are your test cases
dataset_name = "test2 Dataset"
dataset = client.create_dataset(dataset_name, description="Rap battle prompts.")
client.create_examples(
    inputs=[
        {"question": "a rap battle between Atticus Finch and Cicero"},
        {"question": "a rap battle between Barbie and Oppenheimer"},
    ],
    outputs=[
        {"must_mention": ["lawyer", "justice"]},
        {"must_mention": ["plastic", "nuclear"]},
    ],
    dataset_id=dataset.id,
)

# Define AI system
openai_client = wrap_openai(openai.Client())

def predict(inputs: dict) -> dict:
    messages = [{"role": "user", "content": inputs["question"]}]
    response = openai_client.chat.completions.create(messages=messages, model="gpt-3.5-turbo")
    return {"output": response}

# Define evaluators
def must_mention(run: Run, example: Example) -> dict:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return {"key":"must_mention", "score": score}

experiment_results = evaluate(
    predict, # Your AI system
    data=dataset_name, # The data to predict and grade over
    evaluators=[must_mention], # The evaluators to score the results
    experiment_prefix="rap-generator", # A prefix for your experiment names to easily identify them
    metadata={
      "version": "1.0.0",
    },
)

# Temp